In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.3 MB/s eta 0:00:00


In [43]:
import pandas as pd
df = pd.read_csv('/content/sample_data/final_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Depression', axis=1)
y_train = train_set['Depression'].copy()

In [44]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [60]:
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from catboost import CatBoostClassifier as Catboost

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=1000, max_depth=18, n_jobs=-1, random_state=42)),
        ('gbf', GradientBoostingClassifier(max_depth=5, n_estimators=40, random_state=42)),
        ('xgboost', xgb.XGBClassifier(max_depth=6, n_estimators=40, random_state=42)),
        ('catboost',Catboost(iterations=15, depth=6, learning_rate=1, random_state=42))
    ],
    voting='soft'
)

In [61]:
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_train)

0:	learn: 0.2014846	total: 23.9ms	remaining: 334ms
1:	learn: 0.1732878	total: 47.9ms	remaining: 311ms
2:	learn: 0.1676128	total: 72ms	remaining: 288ms
3:	learn: 0.1622520	total: 95ms	remaining: 261ms
4:	learn: 0.1587314	total: 118ms	remaining: 236ms
5:	learn: 0.1564909	total: 139ms	remaining: 209ms
6:	learn: 0.1550266	total: 162ms	remaining: 185ms
7:	learn: 0.1540773	total: 183ms	remaining: 160ms
8:	learn: 0.1532368	total: 205ms	remaining: 136ms
9:	learn: 0.1525108	total: 230ms	remaining: 115ms
10:	learn: 0.1515221	total: 252ms	remaining: 91.7ms
11:	learn: 0.1506706	total: 277ms	remaining: 69.1ms
12:	learn: 0.1500343	total: 298ms	remaining: 45.9ms
13:	learn: 0.1491781	total: 320ms	remaining: 22.9ms
14:	learn: 0.1485353	total: 344ms	remaining: 0us


In [62]:
# accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.9433813077469794

In [63]:
# split target and features
X_test = test_set_from_train.drop('Depression', axis=1)
y_test = test_set_from_train['Depression'].copy()
X_test = scaler.transform(X_test)

In [64]:
y_pred = voting_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9396588486140725

In [65]:
# save final prediction
test_set = pd.read_csv('/content/sample_data/final_test.csv')
del test_set['Depression']
test_set = scaler.transform(test_set)
y_pred = voting_clf.predict(test_set)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Depression': y_pred})
final_prediction.to_csv('final_prediction3.csv', index=False)